### Determine distance between line segments in 2D.

Aims to handle many fringe cases not usually handled by generic linear algebra equations.

Convention: Line segments defined by X,Y values of endpoints.

Exposes intermediary support calculations.

In [2]:
import numpy as np

In [3]:
# First function calculates shortest distance between a point and an infinite line
def point_to_inf_line(x_pt, y_pt, x_line_1, y_line_1, x_line_2, y_line_2):
    dist = (np.abs((y_line_2 - y_line_1) * x_pt - (x_line_2 - x_line_1) * 
                   y_pt + x_line_2 * y_line_1 - y_line_2 * x_line_1) /
            np.sqrt((y_line_2 - y_line_1)**2 + (x_line_2 - x_line_1)**2))
    return(dist)

point_to_inf_line(1,0,2,1,3,1) # returns distance 1.0

1.0

In [4]:
# Calculate minimum distance between point and a line segment
def point_to_line_segment(x_pt, y_pt, x_line_1, y_line_1, x_line_2, y_line_2):
    dist_inf_line = point_to_inf_line(x_pt, y_pt, x_line_1, y_line_1, x_line_2, y_line_2)
    
    # define line from points, in ax + by + c = 0 form 
    # check for vertical line case
    if x_line_1 == x_line_2:
        line_a = 1
        line_b = 0
        line_c = -1 * x_line_1
    else:
        line_m = (y_line_2 - y_line_1) / (x_line_2 - x_line_1) # slope
        line_int = y_line_2 - line_m * x_line_2 # y-intercept
        line_a = -1 * line_m # put in ax + by + c = 0 form
        line_b = 1 # assume equal to 1    
        line_c = -1 * line_int
    
    # find point on line for shortest distance
    x_int = (line_b*(line_b * x_pt - line_a * y_pt) - line_a * line_c) / (line_a ** 2 + line_b ** 2)
    y_int = (line_a * (-1 * line_b * x_pt + line_a * y_pt) - line_b * line_c) / (line_a ** 2 + line_b ** 2)
    
    # if point falls in bounds of line segment, treat as infinite line for calculating distance
    if (x_int >= np.min([x_line_1, x_line_2]) and x_int <= np.max([x_line_1, x_line_2]) and 
        y_int >= np.min([y_line_1, y_line_2]) and y_int <= np.max([y_line_1, y_line_2])):
        return(dist_inf_line)
    else:
        # if nearest point not on segment, use segment endpoints as potential nearest points 
        # return shrtest distance from pt to segment endpoints
        dist_end_pt_1 = np.sqrt((y_line_1 - y_pt)**2 + (x_line_1 - x_pt)**2)
        dist_end_pt_2 = np.sqrt((y_line_2 - y_pt)**2 + (x_line_2 - x_pt)**2)
        return(np.min([dist_end_pt_1, dist_end_pt_2]))
    
point_to_line_segment(1,0,1,1,3,1) # should equal 1

point_to_line_segment(0,0,0,-1,0,1) # equal 0, point falls on line

point_to_line_segment(2, 1, 0, 0, 0, 1) # vertical line

In [105]:
def segment_to_segment(x1, y1, x2, y2, x3, y3, x4, y4):
    
    # center data to origin to avoid precision problems
    # this can be encountered with survey data where points are millions of feet from datum
    min_x = np.min([x1, x2, x3, x4])
    min_y = np.min([y1, y2, y3, y4])
    x1 = x1 - min_x
    x2 = x2 - min_x
    x3 = x3 - min_x
    x4 = x4 - min_x
    y1 = y1 - min_y
    y2 = y2 - min_y
    y3 = y3 - min_y
    y4 = y4 - min_y
    
    # check vertical lines
    if x1 == x2 and x3 == x4: # if both vertical
        # then flip x and y axes, so that slope is not Inf
        min_dist = np.min([point_to_line_segment(y1, x1, y3, x3, y4, x4),
                           point_to_line_segment(y2, x2, y3, x3, y4, x4),
                           point_to_line_segment(y3, x3, y1, x1, y2, x2),
                           point_to_line_segment(y4, x4, y1, x1, y2, x2)])
        return(min_dist)
    # find intersection between the 2 as infinite lines
    elif x1 == x2: # if first vertical
        x_int = x1
        y_int = (y4 - y3)/(x4 - x3)*(x_int - x3) + y3 # point-slope intercept form
    elif x3 == x4: # if second vertical
        x_int = x3
        y_int = (y2 - y1)/(x2 - x1)*(x_int - x1) + y1 # point-slope intercept form
    elif (y2 - y1)/(x2 - x1) == (y4 - y3)/(x4 - x3): # if parallel lines
        min_dist = np.min([point_to_line_segment(x1, y1, x3, y3, x4, y4),
                           point_to_line_segment(x2, y2, x3, y3, x4, y4),
                           point_to_line_segment(x3, y3, x1, y1, x2, y2),
                           point_to_line_segment(x4, y4, x1, y1, x2, y2)])
        return(min_dist)        
    else:
        x_int = (((x1 * y2 - y1 * x2) * (x3 - x4) - (x1 - x2) * (x3*y4 - y3 * x4)) /
                 (x1 - x2) * (y3 - y4) - (y1 - y2) * (x3 - x4))
        y_int = (((x1 * y2 - y1 * x2) * (y3 - y4) - (y1 - y2) * (x3 * y4 - y3 * x4))/
                 ((x1 - x2) * (y3 - y4) - (y1 - y2) * (x3 - x4)))
        
    # check intersection    
    if (x_int >= np.min([x1, x2]) and x_int <= np.max([x1, x2]) and
        x_int >= np.min([x3, x4]) and x_int <= np.max([x3, x4]) and
        y_int >= np.min([y1, y2]) and y_int <= np.max([y1, y2]) and
        y_int >= np.min([y3, y4]) and y_int <= np.max([y3, y4])):
        return(0.0)
    else:
        min_dist = np.min([point_to_line_segment(x1, y1, x3, y3, x4, y4),
                           point_to_line_segment(x2, y2, x3, y3, x4, y4),
                           point_to_line_segment(x3, y3, x1, y1, x2, y2),
                           point_to_line_segment(x4, y4, x1, y1, x2, y2)])
        return(min_dist)

In [99]:
segment_to_segment(0, 0, 0, 1, 1, 0, 1, 1) #vertical, adjacent lines

1.0

In [100]:
segment_to_segment(0, 0, 0, 1, 1, 2, 1, 3) #vertical, offset lines

1.4142135623730951

In [71]:
segment_to_segment(-1, 0, 1, 0, 0, -1, 0, 1) # intersection

0

In [72]:
segment_to_segment(3e6, 3e6, 3e6+1, 3e6+1, 3e6, 3e6+1, 3e6+1, 3e6) # far from axis

0

In [102]:
segment_to_segment(-3e6, -3e6, -3e6+1, -3e6+1, -3e6, -3e6+1, -3e6+1, -3e6) # far from axis, negative

0

In [103]:
segment_to_segment(0,0, 1,1, 1,0, 2,1) # diagonal parallel

0.7071067811865475

In [106]:
segment_to_segment(0,0, 1,1, 2,1, 3,2) # diagonal offset parallel

1.0

In [104]:
segment_to_segment(0,0, 1,1, 1,0, 2,0.9) # diagonal off-parallel

0.7071067811865475

In [108]:
segment_to_segment(0,0, 1,3, 2,3, 3,0) # / \ shape

1.0

In [109]:
segment_to_segment(0,0, 0,3, 2,3, 3,0) # | \ shape

2.0

In [110]:
segment_to_segment(0,0, 0,2, 2,3, 3,0) # | \ shape

2.2135943621178655